In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import glob
import xarray as xr
from params import participants
%matplotlib inline
# %matplotlib widget

In [2]:
participants

['P17ETRPA', 'P18BLAMA']

In [3]:
save = True

In [4]:
def raw_to_da(participant, save = False): # from raw data to one xarray by participant
    print(participant)
    input_file = glob.glob(f"../data/{participant}/*/*.TXT")[0]
    with open(input_file, "r") as file:
        for i in range(2):
            line = file.readline()
    header = json.loads(line[1:])
    meta = header[next(iter(header))]
    srate = meta['sampling rate']
    print('srate :', srate)
    data = np.loadtxt(input_file).T
    print(data.shape)

    if data.shape[0] > 9: # some participants don't have the 5 chans
        eegL = data[5] # A1
        eegR = data[6] # A2
        ecg  = data[7] # A3
        eda  = data[8] # A4
        ppg  = data[9] # A5
    else: 
        eegL = data[5] # A1
        eegR = data[6] # A2
        ecg  = data[7] # A3
        eda  = data[8] # A4
        ppg  = np.zeros((data.shape[1])) # A5 
    
    data_to_array = [eegL , eegR, ecg , eda , ppg]
    time = np.arange(0 , eegL.size / srate,  1 / srate)
    dtypes = ['EEGL','EEGR','ECG','EDA','PPG']
    da = xr.DataArray(data=data_to_array, dims = ['dtype','time'], coords = {'dtype':dtypes, 'time':time}) # xarray 2D (data_type * time)
    if save:
        da.to_netcdf(f'../data_in_da/da_{participant}.nc')
    return da

In [5]:
for participant in participants:
    da = raw_to_da(participant, save=save)
    del da

P17ETRPA
srate : 1000
(11, 4582000)
P18BLAMA
srate : 1000
(11, 2897000)
